In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
data_file_path = "transactions.txt"  

# Read the data from the text file into a list
with open(data_file_path, "r") as file:
    json_data = file.readlines()

# Convert each JSON line into a Python dictionary
data_list = [json.loads(line.strip()) for line in json_data]

# Create a Pandas DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

# Now df contains transaction data in a DataFrame


csv_file = df.to_csv('transaction.csv', index=False)
df = pd.read_csv('transaction.csv')

columns_to_drop = ['echoBuffer', 'merchantCity', 'merchantState', 'merchantZip', 'posOnPremises', 'recurringAuthInd']
# Drop the specified columns with 0 non-null values in place
df.drop(columns=columns_to_drop, inplace=True)

df.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud
0,737265056,737265056,5000.0,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,2.0,...,2015-03-14,2015-03-14,414,414,1803,PURCHASE,0.0,False,False,False
1,737265056,737265056,5000.0,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,9.0,...,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.0,True,False,False
2,737265056,737265056,5000.0,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,9.0,...,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.0,False,False,False
3,737265056,737265056,5000.0,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,US,9.0,...,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.0,False,False,False
4,830329091,830329091,5000.0,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,2.0,...,2015-08-06,2015-08-06,885,885,3143,PURCHASE,0.0,True,False,False


### Reversed Transactions

In [4]:
reversed_transactions = df[(df['transactionType'] == 'REVERSAL')]
reversed_transactions.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud
39,574788567,574788567,2500.0,2200.98,2016-05-24T01:38:03,215.13,Convenient Tire,US,US,9.0,...,2015-10-13,2015-10-13,206,206,8522,REVERSAL,299.02,True,False,False
73,574788567,574788567,2500.0,2432.72,2016-10-07T10:23:57,3.87,Eazy Repair,US,US,5.0,...,2015-10-13,2015-10-13,579,579,4219,REVERSAL,67.28,True,False,False
101,924729945,924729945,50000.0,49831.43,2016-10-19T14:01:45,168.57,discount.com,US,US,5.0,...,2014-07-25,2014-07-25,205,265,9459,REVERSAL,168.57,False,False,False
133,984504651,984504651,50000.0,46367.41,2016-01-16T09:53:15,450.74,Planet Fitness #849960,US,US,5.0,...,2015-07-27,2015-07-27,640,640,8332,REVERSAL,3632.59,False,False,False
156,984504651,984504651,50000.0,41909.30,2016-01-25T20:39:15,81.73,AMC #724446,US,US,9.0,...,2015-07-27,2015-07-27,640,640,8332,REVERSAL,8090.70,True,False,False


In [15]:
total_reversed_dollar_amount = reversed_transactions['transactionAmount'].sum()

print("Total Number of Reversed Transactions:", len(reversed_transactions))
print("Total Dollar Amount for Reversed Transactions:", total_reversed_dollar_amount)

Total Number of Reversed Transactions: 20303
Total Dollar Amount for Reversed Transactions: 2821792.5


In [36]:
fraud_reversals = df[(df['transactionType'] == 'REVERSAL') & (df['isFraud'] == True)]

num_fraud_reversals = len(fraud_reversals)

print("Number of fraudulent reversal transactions:", num_fraud_reversals)

Number of fraudulent reversal transactions: 337


### Multi-Swipe Transactions

To identify multi-swipes, I checked for:  
- Multiple transactions for the same account within a short time span (let's say a few minutes).  
- The same transaction amount.  
- The same merchant.

I have also assumed that multi-swipe transactions take place in a short time say 15 minutes.

In [24]:
df['transactionDateTime'] = pd.to_datetime(df['transactionDateTime'])

# Sort by accountNumber, merchantName, and transactionDateTime
df = df.sort_values(by=['accountNumber', 'merchantName', 'transactionDateTime'])

# Find potential multi-swipes
df['is_potential_multiswipe'] = ((df['accountNumber'] == df['accountNumber'].shift(-1)) &
                                (df['merchantName'] == df['merchantName'].shift(-1)) &
                                (df['transactionAmount'] == df['transactionAmount'].shift(-1)) &
                                ((df['transactionDateTime'].shift(-1) - df['transactionDateTime']).dt.total_seconds() < 15*60))

# Filter for potential multi-swipe transactions
potential_multiswipes = df[df['is_potential_multiswipe']]

potential_multiswipes.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,is_potential_multiswipe
541963,100088067,100088067,50000.0,49834.20,2016-11-20 07:57:05,22.32,AMC #79863,US,US,9.0,...,2014-01-09,546,546,4615,PURCHASE,165.80,True,False,False,True
541955,100088067,100088067,50000.0,49686.44,2016-10-16 18:01:00,411.35,Fresh Flowers,US,US,2.0,...,2014-01-09,402,402,2077,PURCHASE,313.56,False,False,False,True
87919,100328049,100328049,5000.0,4648.84,2016-01-15 20:34:35,43.74,Lyft,US,US,9.0,...,2014-01-07,206,206,2941,PURCHASE,351.16,False,False,False,True
151517,100737756,100737756,250.0,221.23,2016-07-10 14:31:07,43.25,34th BBQ #166379,US,US,2.0,...,2015-06-21,463,463,4317,PURCHASE,28.77,True,False,False,True
151426,100737756,100737756,250.0,55.59,2016-01-18 01:55:24,693.50,Franks Deli,US,US,5.0,...,2015-06-21,463,463,4317,PURCHASE,194.41,True,False,False,True


In [35]:
total_multiswipe_dollar_amount = potential_multiswipes['transactionAmount'].sum()

print("Total Number of Multi Swipe Transactions:", len(potential_multiswipes))
print("Total Dollar Amount for Multi Swipe Transactions:", total_multiswipe_dollar_amount)

Total Number of Multi Swipe Transactions: 13390
Total Dollar Amount for Multi Swipe Transactions: 1932091.4


In [37]:
fraud_multiswipes = df[(df['is_potential_multiswipe']) & (df['isFraud'])]

num_fraud_multiswipes = len(fraud_multiswipes)

print("Number of fraudulent multi-swipe transactions:", num_fraud_multiswipes)

Number of fraudulent multi-swipe transactions: 233


### Observations

### Reversed Transactions:

- Frequency: Reversed transactions might occur relatively infrequently compared to regular purchases.

- Transaction Amounts: Reversed transactions are likely to have the same transaction amount as the original purchase.

- Merchant Relationships: Reversed transactions may reveal patterns related to specific merchants or merchant categories that are more prone to chargebacks or refunds.

- Timing: Investigating the time interval between the original purchase and the reversal can provide insights into how quickly reversals are processed.

### Multi-Swipe Transactions:

- Frequency: Multi-swipe transactions may occur as isolated incidents or in patterns, depending on the nature of the problem (e.g., technical issue at a specific merchant).

- Transaction Amounts: Multi-swipe transactions are likely to have identical transaction amounts since they represent multiple charges for the same purchase.

- Merchant Relationships: Patterns may emerge when examining which merchants are involved in multi-swipe incidents. This could be indicative of technical issues or misunderstandings between customers and merchants.

- Timing: Analyzing the time intervals between multiple swipes can reveal whether they occurred in rapid succession or were spread out over time.

